##Environment
Creating the environment in order to get the sequences of actions, rewards and next states.

In [1]:
import sys
import random
import numpy as np
def weighted_choice(v, p):
   total = sum(p)
   r = random.uniform(0, total)
   upto = 0
   for c, w in zip(v,p):
      if upto + w >= r:
         return c
      upto += w
   assert False, "Shouldn't get here"

In [2]:
class MDP_Environment:
    def __init__(self, transition_probs, rewards, initial_state='s0'):
        self._transition_probs = transition_probs
        self._rewards = rewards
        self._initial_state = initial_state
        self.n_states = len(transition_probs)
        self.reset()

    def get_all_states(self):
        """ return a tuple of all possible states """
        return tuple(self._transition_probs.keys())

    def get_possible_actions(self, state):
        """ return a tuple of possible actions in a given state """
        return tuple(self._transition_probs.get(state, {}).keys())

    def is_terminal(self, state):
        """ return True if state is terminal or False if it isn't """
        return len(self.get_possible_actions(state)) == 0

    def get_next_states(self, state, action):
        """ return a dictionary of {next_state1 : P(next_state1 | state, action), next_state2: ...} """
        assert action in self.get_possible_actions(state), "cannot do action %s from state %s" % (action, state)
        return self._transition_probs[state][action]

    def get_transition_prob(self, state, action, next_state):
        """ return P(next_state | state, action) """
        return self.get_next_states(state, action).get(next_state, 0.0)

    def get_reward(self, state, action, next_state):
        """ return the reward you get for taking action in state and landing on next_state"""
        assert action in self.get_possible_actions(state), "cannot do action %s from state %s" % (action, state)
        return self._rewards.get(state, {}).get(action, {}).get(next_state, -0.01)

    def reset(self):
        """ reset the game, return the initial state"""
        if self._initial_state is None:
            self._current_state = random.choice(tuple(self._transition_probs.keys()))
        elif self._initial_state in self._transition_probs:
            self._current_state = self._initial_state
        elif callable(self._initial_state):
            self._current_state = self._initial_state()
        else:
            raise ValueError("initial state %s should be either a state or a function() -> state" % self._initial_state)
        return self._current_state

    def step(self, action):
        """ take action, return next_state, reward, is_done, empty_info """
        possible_states, probs = zip(*self.get_next_states(self._current_state, action).items())
        next_state = weighted_choice(possible_states, p=probs)
        reward = self.get_reward(self._current_state, action, next_state)
        is_done = self.is_terminal(next_state)
        self._current_state = next_state
        return next_state, reward, is_done, {}

    def render(self):
        if(self._current_state=='s3'):
          print("Currently at goal state %s" % self._current_state)
        if(self._current_state=='s7'):
          print("Currently at hole state %s" % self._current_state)
        if(self._current_state!='s3' and self._current_state!='s7'):
          print("Currently at %s" % self._current_state)

left = a0

up = a1

right = a2

down = a3

In [4]:
# creating dictionaries for transition probabilities and transition rewards as given in the problem statement
transition_probs = {'s0' : {'a0' : {'s0' : 0.9, 's4' : 0.1}, 'a1' : {'s0' : 0.9, 's1' : 0.1}, 'a2' : {'s1' : 0.8, 's0' : 0.1, 's4' : 0.1}, 'a3' : {'s4' : 0.8, 's0' : 0.1, 's1' : 0.1}},
                    's1' : {'a0' : {'s1' : 0.2, 's0' : 0.8}, 'a1' : {'s0' : 0.1, 's1' : 0.8, 's2' : 0.1}, 'a2' : {'s2' : 0.8, 's1' : 0.2}, 'a3' : {'s1' : 0.8, 's0' : 0.1, 's2' : 0.1}},
                    's2' : {'a0' : {'s1' : 0.8, 's2' : 0.1, 's6' : 0.1}, 'a1' : {'s2' : 0.8, 's1' : 0.1, 's3' : 0.1}, 'a2' : {'s3' : 0.8, 's2' : 0.1, 's6' : 0.1}, 'a3' : {'s6' : 0.8, 's3' : 0.1, 's1' : 0.1}},
                    's3' : {},
                    's4' : {'a0' : {'s4' : 0.8, 's0' : 0.1, 's8' : 0.1}, 'a1' : {'s0' : 0.8, 's4' : 0.2}, 'a2' : {'s4' : 0.8, 's0' : 0.1, 's8' : 0.1}, 'a3' : {'s8' : 0.8, 's4' : 0.2}},
                    's5' : {},
                    's6' : {'a0' : {'s6' : 0.8, 's2' : 0.1, 's10' : 0.1}, 'a1' : {'s2' : 0.8, 's6' : 0.1, 's7' : 0.1}, 'a2' : {'s7' : 0.8, 's2' : 0.1, 's10' : 0.1}, 'a3' : {'s10' : 0.8, 's6' : 0.1, 's7' : 0.1}},
                    's7' : {},
                    's8' : {'a0' : {'s8' : 0.9, 's4' : 0.1}, 'a1' : {'s4' : 0.8, 's8' : 0.1, 's9' : 0.1}, 'a2' : {'s9' : 0.8, 's4' : 0.1, 's8' : 0.1}, 'a3' : {'s8' : 0.9, 's9' : 0.1}},
                    's9' : {'a0' : {'s8' : 0.8, 's9' : 0.2}, 'a1' : {'s9' : 0.8, 's8' : 0.1, 's10' : 0.1}, 'a2' :{'s10' : 0.8, 's9' : 0.2}, 'a3' : {'s9' : 0.8, 's10' : 0.1, 's8' : 0.1}},
                    's10' : {'a0' : {'s9' : 0.8, 's6' : 0.1, 's10' : 0.1}, 'a1' : {'s6' : 0.8, 's9' : 0.1, 's11' : 0.1}, 'a2' : {'s11' : 0.8, 's6' : 0.1, 's10' : 0.1}, 'a3' : {'s10' : 0.8, 's11' : 0.1, 's9' : 0.1}},
                    's11' : {'a0' : {'s10' : 0.8, 's7' : 0.1, 's11' : 0.1}, 'a1' : {'s7' : 0.8, 's10' : 0.1, 's11' : 0.1}, 'a2' : {'s11' : 0.9, 's7' : 0.1}, 'a3' : {'s11' : 0.9, 's10' : 0.1}}}

rewards = {'s2' : {'a1' : {'s3' : 1}, 'a2' : {'s3' : 1}, 'a3' : {'s3' : 1}},
           's6' : {'a1' : {'s7' : -1}, 'a2' : {'s7' : -1}, 'a3' : {'s7' : -1}},
           's11' : {'a0' : {'s7' : -1}, 'a1' : {'s7' : -1}, 'a2' : {'s7' : -1}}}

In [5]:
env = MDP_Environment(transition_probs, rewards)

For both the policy iteration and value iteration algorithms implemented below the value corresponding to state 5 is set to 0 as a default value.
While the action corresponding to state 5 as well as terminal state is set to None.

## Policy Iteration
In policy iteration, we start by choosing an arbitrary policy $\boldsymbol{\pi}$. Then, we iteratively evaluate and improve the policy until convergence.
We evaluate a policy $\pi(s)$ by calculating the state value function
V(s):

$V(s) = \sum_{s',r'}p(s', r|s,\pi(s))[r+\gamma V(s')]$

Then, we calculate the improved policy by using one-step look-ahead to replace the initial policy $\pi(s)$:

$\pi(s) = arg\max_{a} \sum_{s',r'} p(s', r|s,a)[r+\gamma V(s')]$

In the beginning, we don’t care about the initial policy \pi_0 being optimal or not. During the execution, we concentrate on improving it on every iteration by repeating policy evaluation and policy improvement steps. Using this algorithm we produce a chain of policies, where each policy is an improvement over the previous one:

  $\pi_0 \xrightarrow[]{\text{E}} v_{\pi_0} \xrightarrow[]{\text{I}} \pi_1 \xrightarrow[]{\text{E}} v_{\pi_1} \xrightarrow[]{\text{I}} \pi_2 \xrightarrow[]{\text{E}} \dotsi \xrightarrow[]{\text{I}} \pi_* \xrightarrow[]{\text{E}} v_{*}$

In [6]:
states = list(env.get_all_states())
# choosing a random action space for each state
action_space = []
for i in range(len(states)):
  actions = env.get_possible_actions(states[i])
  if(len(actions)>0):
    act = random.choice(actions)
  else:
    act = 'None'
  action_space.append(act)
print(action_space)

['a2', 'a0', 'a0', 'None', 'a1', 'None', 'a1', 'None', 'a2', 'a0', 'a0', 'a1']


In [7]:
# initializing the value function for the current action space to a vector of zeros
v_k = []
for i in range(len(states)):
  v_k.append(0)

In [8]:
#loop for policy improvement
flag1 = 1
num_steps = 0
while(flag1==1):
  num_steps += 1
  #loop for policy evaluation
  flag2 = 1
  while(flag2==1):
    v_kplus1 = []
    for i in range(len(states)):
      if(action_space[i]!='None'):
        next_states = list(env.get_next_states(states[i], action_space[i]).keys())
        sum = 0
        for j in range(len(next_states)):
          sum = env.get_transition_prob(states[i], action_space[i], next_states[j])*(env.get_reward(states[i], action_space[i], next_states[j]) + v_k[states.index(next_states[j])]) + sum
        v_kplus1.append(sum)
      else:
        v_kplus1.append(0)
    max = -1
    for k in range(len(states)):
      if(abs(v_kplus1[k]-v_k[k])>max):
        max = abs(v_kplus1[k]-v_k[k])
    if(max<=0.01):
      flag2 = 0
    v_k = v_kplus1
  action_space_dash = []
  for k in range(len(states)):
    if(action_space[k]!='None'):
      actions = env.get_possible_actions(states[k])
      max = -1
      max_index = 0
      for act in range(len(actions)):
        next_states = list(env.get_next_states(states[k], actions[act]).keys())
        sum = 0
        for j in range(len(next_states)):
          sum = env.get_transition_prob(states[k], actions[act], next_states[j])*(env.get_reward(states[k], actions[act], next_states[j]) + v_kplus1[states.index(next_states[j])]) + sum
        if(sum>max):
          max = sum
          max_index = act
      action_space_dash.append(actions[max_index])
    else:
      action_space_dash.append('None')
  num = 0
  for i in range(len(states)):
    if(action_space[i]==action_space_dash[i]):
      num +=1
  if(num==12):
    flag1 = 0
  action_space = action_space_dash
print('Number of steps required in order to obtain the optimal policy and value function is', num_steps)
print('Optimal Policy - ', action_space)
print('Optimal Value Function - ', v_kplus1)

Number of steps required in order to obtain the optimal policy and value function is 7
Optimal Policy -  ['a2', 'a2', 'a2', 'None', 'a1', 'None', 'a0', 'None', 'a1', 'a0', 'a0', 'a3']
Optimal Value Function -  [0.9551744405284887, 0.9701896255900637, 0.9833257236758887, 0, 0.9416532885684301, 0, 0.8740149975680315, 0, 0.9261864677278419, 0.9123110204249538, 0.8953842793185613, 0.7087918791639867]


## Value Iteration
In value iteration, we compute the optimal state value function by iteratively updating the estimate $\textbf{v(s)}$. We start with a random value function V(s). At each step, we update it:

  $V(s) = \max_{a} \sum_{s',r'}p(s', r|s,a)[r+\gamma V(s')]$

Hence, we look-ahead one step and go over all possible actions at each iteration to find the maximum.

The update step is very similar to the update step in the policy iteration algorithm. The only difference is that we take the maximum over all possible actions in the value iteration algorithm.

Instead of evaluating and then improving, the value iteration algorithm updates the state value function in a single step. This is possible by calculating all possible rewards by looking ahead.

In [9]:
v_k = []
for i in range(len(states)):
  v_k.append(0)
flag = 1
num = 0
while(flag==1):
  num += 1
  v_kplus1 = []
  for i in range(len(states)):
    actions = env.get_possible_actions(states[i])
    if(len(actions)!=0):
      max = -10
      for j in range(len(actions)):
        next_states = list(env.get_next_states(states[i], actions[j]).keys())
        sum = 0
        for k in range(len(next_states)):
          sum = env.get_transition_prob(states[i], actions[j], next_states[k])*(env.get_reward(states[i], actions[j], next_states[k]) + v_k[states.index(next_states[k])]) + sum
        if(sum>=max):
          max = sum
      v_kplus1.append(max)
    else:
      v_kplus1.append(0)
  max = -1
  for k in range(len(states)):
    if(abs(v_kplus1[k]-v_k[k])>max):
      max = abs(v_kplus1[k]-v_k[k])
  if(max<=0.01):
    flag = 0
  v_k = v_kplus1

action_space = []
for i in range(len(states)):
  actions = env.get_possible_actions(states[i])
  if(len(actions)!=0):
    max = -10
    for j in range(len(actions)):
      next_states = list(env.get_next_states(states[i], actions[j]).keys())
      sum = 0
      for k in range(len(next_states)):
        sum = env.get_transition_prob(states[i], actions[j], next_states[k])*(env.get_reward(states[i], actions[j], next_states[k]) + v_k[states.index(next_states[k])]) + sum
      if(sum>=max):
        max = sum
        act = actions[j]
    action_space.append(act)
  else:
    action_space.append('None')
print('Number of steps required in order to obtain the optimal policy and value function is', num)
print('Optimal Policy - ', action_space)
print('Optimal Value Function - ', v_kplus1)

Number of steps required in order to obtain the optimal policy and value function is 20
Optimal Policy -  ['a2', 'a2', 'a2', 'None', 'a1', 'None', 'a0', 'None', 'a1', 'a0', 'a0', 'a3']
Optimal Value Function -  [0.9528669451625345, 0.9683343099109085, 0.981806266398715, 0, 0.9389091676094532, 0, 0.8625371741259842, 0, 0.9229562493380312, 0.9086724176404589, 0.8903134406019679, 0.7051207359650054]


## Comparison between Policy Iteration and Value Iteration in terms of Convergence

Both the dynamic programming based algorithms are guaranteed to converage to an optimal policy, which for our environment comes out to be the **same** for both algorithms.
Policy iteration method converges in a fewer number of iterations(4-5) of the outer loop although each outer loop requires an inner loop of value evaluation untill convergence with an error of 1 percent while value iteration method takes significantly higher steps(around 20) to converge compared to policy iteration.
But policy iteration takes more time compared to value iteration for our MDP problem as there is an additional value evaluation step involved in policy iteration but as our action space is limited to 4 actions for every state value iteration takes less time. For more complicated action spaces it might take longer to converge compared to policy iteration.